In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/how-to-get-32gb-ram/train_labels.parquet
/kaggle/input/how-to-get-32gb-ram/__results__.html
/kaggle/input/how-to-get-32gb-ram/train.parquet
/kaggle/input/how-to-get-32gb-ram/submission.csv
/kaggle/input/how-to-get-32gb-ram/__notebook__.ipynb
/kaggle/input/how-to-get-32gb-ram/__output__.json
/kaggle/input/how-to-get-32gb-ram/custom.css
/kaggle/input/northen/submission_template.csv
/kaggle/input/northen/train.csv
/kaggle/input/northen/test.csv


In [2]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install sacremoses
!pip install sacrebleu
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 5.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 9.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.9 MB/s eta 0:00:00


In [3]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import MarianTokenizer, MarianMTModel, Trainer, TrainingArguments

2024-08-03 19:18:04.360173: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 19:18:04.360314: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 19:18:04.494808: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
train = pd.read_csv("/kaggle/input/northen/train.csv")

In [5]:
train

,ID,Northern,Central
0,0,มะ ใด จะ เตี้ยง ใคร่ อยาก กิ๋น เข้า เข้า เจ๊า ...,เมื่อ ไหร่ จะ เที่ยง หิว ข้าว ข้าว เช้า วัน นี...
1,1,กิ๋น น้ำ นัก นัก เน้อ ไป ดี กิ๋น น้ำ น้อย บ่า ...,ดื่ม น้ำ เยอะ เยอะ นะ อย่า ดื่ม น้ำ น้อย ไม่ ง...
2,2,ใน วัน ปาก ปี๋ จะ เอา บะ หนุน มา เยียะ กิ๋น เป...,ใน วัน ปาก ปี จะ นำ ขนุน มา ทำ อา หาร โบ ราณ เ...
3,3,เอา จะ ใด ดี วัน พูก ครู หื้อ ส่ง ก๋าน บ้าน แล...,เอา ยัง ไง ดี วัน พรุ่ง นี้ ครู ให้ ส่ง การ บ้...
4,4,วัน พูก เดือน สิบ สอง เป็ง แม่ จะ แก๋ง ปี๋ ต๋า...,พรุ่ง นี้ ตรง กับ วัน สารท เดือน สิบ แม่ จะ แก...
...,...,...,...
1595,1595,โฮง เฮียน โทร มา บอก ว่า คิง ติด ศูนย์ ก๊ะ แหม...,โรง เรียน โทร มา บอก ว่า แก ติด ศูนย์ เหรอ แล้...
1596,1596,กลุ่ม จาติพันธุ์ ตี้ ฮ้อง ว่า ละ ว้า แต่ เก๊า ...,กลุ่ม ชาติพันธุ์ หนึ่ง ที่ เรียก ว่า ละ ว้า เด...
1597,1597,ตั๋ว เขา ฮู้ ก่อ โกม ลอย ตี้ เขา ชอบ จิ๊ ปล่อย...,พวก เธอ รู้ ไหม โคม ลอย ที่ เขา จุด กัน ตอน งา...
1598,1598,อ้าย คำ ไป เอา ไม้ ยู ปื๊น กะ ล่าง บ้าน มา หื้...,พี่ คำ ไป เอา ไม้ กวาด ใต้ ถุน บ้าน ให้ หน่อย ...


In [6]:
train_set,val_set = train_test_split(train,test_size=0.1 ,random_state=42)

In [7]:
train_set['Northern'].to_csv('source_lang_train.txt', sep='\t', index=False, header=False)
train_set['Central'].to_csv('target_lang_train.txt', sep='\t', index=False, header=False)

In [8]:
val_set['Northern'].to_csv('source_lang_val.txt', sep='\t', index=False, header=False)
val_set['Central'].to_csv('target_lang_val.txt', sep='\t', index=False, header=False)

In [9]:
class TranslationDataset(Dataset):
    def __init__(self, source_file, target_file, tokenizer, max_length=128):
        self.source_texts = open(source_file, 'r').readlines()
        self.target_texts = open(target_file, 'r').readlines()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source_text = self.source_texts[idx].strip()
        target_text = self.target_texts[idx].strip()
        source_tokenized = self.tokenizer(source_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        target_tokenized = self.tokenizer(target_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")

        source_tokenized = {key: val.squeeze() for key, val in source_tokenized.items()}
        target_tokenized = {key: val.squeeze() for key, val in target_tokenized.items()}

        return {
            'input_ids': source_tokenized['input_ids'],
            'attention_mask': source_tokenized['attention_mask'],
            'labels': target_tokenized['input_ids']
        }

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments

In [144]:
model_name = 'Helsinki-NLP/opus-mt-mul-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


In [145]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(64172, 512, padding_idx=64171)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(64172, 512, padding_idx=64171)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [148]:
train_dataset = TranslationDataset('source_lang_train.txt', 'target_lang_train.txt', tokenizer)
val_dataset = TranslationDataset('source_lang_val.txt', 'target_lang_val.txt', tokenizer)

In [149]:
torch.cuda.empty_cache()

In [200]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=80,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps = 4,
    eval_accumulation_steps = 8,
    warmup_steps=1000,
    weight_decay=0.5,
    learnning_rate = 5e-5,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [201]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()


Epoch,Training Loss,Validation Loss
0,No log,0.695796
1,0.014000,0.695247
2,0.014000,0.694509
4,0.014100,0.693703
5,0.013300,0.693586
6,0.013000,0.693813
8,0.012800,0.694852
9,0.012800,0.695262
10,0.012800,0.695175
12,0.012500,0.694944


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[64171]], 'forced_eos_token_id': 0}


TrainOutput(global_step=400, training_loss=0.009715877417474985, metrics={'train_runtime': 1238.7955, 'train_samples_per_second': 92.994, 'train_steps_per_second': 0.323, 'total_flos': 3396344978866176.0, 'train_loss': 0.009715877417474985, 'epoch': 69.56521739130434})

In [202]:
model.save_pretrained('./results/final_model')

tokenizer.save_pretrained('./results/final_model')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[64171]], 'forced_eos_token_id': 0}


('./results/final_model/tokenizer_config.json',
 './results/final_model/special_tokens_map.json',
 './results/final_model/vocab.json',
 './results/final_model/added_tokens.json')

In [203]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


In [204]:
def translate_text(input_text, model, tokenizer, max_length=128, device='cuda'):
    model.to(device)
    inputs = tokenizer(input_text, return_tensors="pt", padding='max_length', truncation=True, max_length=max_length).to(device)
    translated_tokens = model.generate(**inputs, max_length=max_length, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

In [205]:
input_sentence = "อ้าย อ้าย วัน พูก อ้าย จะ ไป ไหน ก่อ น้อง ว่า จะ ขอ ไป ส่ง ซื้อ คัว ตี้ กาด หลวง กำ อี้ แหนะ"
translated_sentence = translate_text(input_sentence, model, tokenizer)
print(f"Translated sentence: {translated_sentence}")

Translated sentence: พี่ พี่ พรุ่ง นี้ พี่ จะ ไป ไหน ไหม น้อง ว่า จะ ขอ ไป ส่ง ซื้อ ของ ที่ ตลาด วโรรส หน่อย นะ


In [206]:
sample = pd.read_csv('/kaggle/input/northen/submission_template.csv')

In [207]:
sample

,ID,Central
0,0,ประ โยค ไทย กลาง
1,1,ประ โยค ไทย กลาง
2,2,ประ โยค ไทย กลาง
3,3,ประ โยค ไทย กลาง
4,4,ประ โยค ไทย กลาง
...,...,...
395,395,ประ โยค ไทย กลาง
396,396,ประ โยค ไทย กลาง
397,397,ประ โยค ไทย กลาง
398,398,ประ โยค ไทย กลาง


In [208]:
torch.cuda.empty_cache()

In [209]:
from tqdm import tqdm

In [210]:
model_path = './results/final_model'
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

test_df = pd.read_csv('/kaggle/input/northen/test.csv')

def translate_text(input_text, model, tokenizer, max_length=128, device='cuda'):
    model.to(device)
    inputs = tokenizer(input_text, return_tensors="pt", padding='max_length', truncation=True, max_length=max_length).to(device)
    translated_tokens = model.generate(**inputs, max_length=max_length, num_beams=4, early_stopping=True)
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

pred = []
for text in tqdm(test_df['Northern'], desc="Translating"):
    pred.append(translate_text(text, model, tokenizer))


Translating: 100%|██████████| 400/400 [03:17<00:00,  2.02it/s]


In [211]:
print(pred)

['วัน นี้ มี ประ ชุม หัว หน้า บอก ว่า ต้อง เข้า ทุก คน ท่าน มี เรื่อง สํา คัญ จะ ประ กาศ', 'ยาย พิศ เขา เป็น คน ดี นุ่ง ขาว ห่ม ขาว เข้า วัด อยู่ อย่า เห็น บ่อย เวลา ไป วัด ตอน วัน พระ', 'แฟชั่น แบบ ไทย ไทย เรา พอ นาง แบบ เดิน ออก มา คน เขา ก็ ดู ที่ หน้า ตา ก่อน เสื้อ ผ้า มา อัน ดับ สอง', 'หนู ว่า จะ ไป รับ ช้าง ขาย ของ ที่ ตลาด เย็น จะ ได้ ช่วย พี่ หา เงิน บ้าง ดี ไหม', 'ไม่ ว่า ผู้ หญิง ไม่ ว่า ผู้ ชาย ตอน นี้ ขี่ รถ กัน เปล่าน ปาด โทะ ขี่ ไป เป็น พวก พล่าน ได้ พัด กัน สัก วัน น่ะ', 'บาง คน ไป ใช้ บริการ ของ โรง แรม ห้า ดาว ก็ ชอบ ที่ จะ หา ร่อย ฟ้อง ร้อง เอา แต่ เสีย หาย อะ ไร ให้ ได้ บ้าง น่ะ', 'ช่วง เดือน แปด เข้า เดือน เก้า เหนือ ที่ วัด เจดีย์ หลวง ท่าน มี งาน ใส่ ขัน ดอก อินทขีล เรา ไป เที่ยว กัน สัก หน่อย นะ', 'ไป ไหว้ ป้า ละ ยัง รีบ ไป ไหว้ เผื่อ ป้า ให้ เงิน ซัก ร้อย สอง ร้อย เป็น ค่า ขนม', 'แม่ ไม่ ค่อย จะ ได้ อยู่ บ้าน นะ ช่วง นี้ ของ ผ้า ของ ผ้า ดูน เต็ม ซ้า หมด ลูก สัก ให้ แม่ หน่อย นะ', 'มัคนายก หา งาน นี้ ก็ เหนื่อย พอ ประ มาณ แล้ว ยัง ต้อง มา หา ที่ อยู่ อีก จะ เอา อ

In [212]:
sample['Central'] = pred

In [213]:
sample

,ID,Central
0,0,วัน นี้ มี ประ ชุม หัว หน้า บอก ว่า ต้อง เข้า ...
1,1,ยาย พิศ เขา เป็น คน ดี นุ่ง ขาว ห่ม ขาว เข้า ว...
2,2,แฟชั่น แบบ ไทย ไทย เรา พอ นาง แบบ เดิน ออก มา ...
3,3,หนู ว่า จะ ไป รับ ช้าง ขาย ของ ที่ ตลาด เย็น จ...
4,4,ไม่ ว่า ผู้ หญิง ไม่ ว่า ผู้ ชาย ตอน นี้ ขี่ ร...
...,...,...
395,395,หยุด เอี่ยม ป่า เที่ยว ภู เขา กลาง ขนาด นี้ มา...
396,396,เอา เงิน พวง มา ลัย ไป หนึ่ง ร้อย บาท เอา ไว้ ...
397,397,อย่า เพิ่ง กิ่ง ไม่ มุด แรง ก็ เดี๋ยว ลูก ที่ ...
398,398,พี่ ค้า ซื้อ ของ แล้ว นี่ ได้ ปลา ดุก ปิ้ง ตํา...


In [214]:
sample.to_csv('submission10.csv',index=False)